In [1]:
# Importando bibliotecas

import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('attacks.csv', encoding='latin-1')

In [3]:
pd.options.display.max_columns = 25

df.head()

# pdf, href formula, href são apenas documentação e não serão úteis na análise
# É necessário investigar as diferenças entre Case number, Case number.1 e Case Number.2 para poder decidir sua utilidade
# Investigar Unnamed: 22 e Unnamed: 23

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [4]:
df.info()

# Unnamed: 22 e Unnamed: 23 quase não tem registros, apenas checar o que são e, muito provavelmente droppar
# Checar se os não-nulos estão todos no final do documento

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [5]:
df.loc[~df['Unnamed: 22'].isnull(),'Unnamed: 22']

#Nada relevante na coluna "Unnamed: 22", será droppada

1478    stopped here
Name: Unnamed: 22, dtype: object

In [6]:
df.loc[~df['Unnamed: 23'].isnull(),'Unnamed: 23']

#Nada relevante na coluna "Unnamed: 23", será droppada

4415             Teramo
5840    change filename
Name: Unnamed: 23, dtype: object

In [7]:
df.loc[df['Case Number'].isnull(),:]

# Aparentemente, não há mais dados após a linha 8702

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5488,NaN,Reported 06-Sep-1905,1905.0,Provoked,USA,Florida,"Fort Pierce, St Lucie County",Fishing,David Curry,M,NaN,Lacerations to leg from hooked shark PROVOKED ...,N,NaN,NaN,"Muncie Evening Press, 8/6/1913",1905.09.06.R-Curry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.09.06.R,1905.09.06.R,815.0,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df['Case Number'].value_counts()

# Aparentemente Case Number possui 8702 registros enquanto os demais possue 6302, porque essa coluna possui 2400 zeros.

0               2400
2014.08.02         2
1962.06.11.b       2
2012.09.02.b       2
2005.04.06         2
                ... 
1889.01.31.R       1
1847.00.00.b       1
1935.08.26         1
ND.0005            1
1985.07.19         1
Name: Case Number, Length: 6287, dtype: int64

In [9]:
print(df['Case Number'].isnull().sum())
print(df.loc[8702:25721,'Case Number'].isnull().sum())

# Fortes indícios de que a partir da linha 8702, todas estão vazias

17021
17020


In [10]:
df.loc[df['Case Number']=='0',:]

# Praticamente confirmado que as linhas entre 6302 e 8701 possuem Case Number 0 e nulo nas demais colunas

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0,NaN,NaN
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8697,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.loc[df['Year'].isnull(),:]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
187,2017.01.08.R,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,NaN,Spearfishing,Kerry Daniel,M,35,"No attack, shark made a threat display",NaN,NaN,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6116.0,NaN,NaN
6079,1836.08.19.R,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1836.08.19.R,1836.08.19.R,224.0,NaN,NaN
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#   Conclusões até agora:
# - Colunas Unnamed: 22 e Unnamed: 23 não possuem informações e serão droppadas
# - Colunas pdf, href formula, href são apenas documentação dos casos e não serão úteis na análise
# - A partir da linha 8702 todos os registros estão vazios
# - Entre a linha 6302 e 8701, Case Number está preenchido com 0 e as demais colunas estão vazias

In [13]:
# Inciaremos a limpeza da base selecionando as colunas que interessam para base de acordo com as conclusões acima

df.columns
selected_columns=['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source','Case Number.1', 'Case Number.2']

In [325]:
# Criando uma versão da base sem as colunas mencionadas acima e com as linhas vazias

df.columns
df2=df.loc[0:6301,selected_columns]

In [326]:
# - Checando a diferença entre as colunas de Case Number, Case Number.1 e Case Number.2

(~(df2['Case Number']==df2['Case Number.1'])).sum()

24

In [327]:
(~(df2['Case Number']==df2['Case Number.2'])).sum()

4

In [328]:
(~(df2['Case Number.1']==df2['Case Number.2'])).sum()

20

In [329]:
# Ao identificar que há apenas 24 diferentes entre Case Number e Case Number.1 e menos ainda nas outras comparações,
# é feita uma inspeção de cada diferença

df2.loc[~(df2['Case Number']==df2['Case Number.1']),['Case Number','Case Number.1','Case Number.2']]

,Case Number,Case Number.1,Case Number.2
34,2018.04.03,2018.04.02,2018.04.03
117,2017.07.20.a,2017/07.20.a,2017.07.20.a
144,2017.05.06,2017.06.06,2017.05.06
217,2016.09.15,2016.09.16,2016.09.15
314,2016.01.24.b,2015.01.24.b,2016.01.24.b
334,2015.12.23,2015.11.07,2015.12.23
339,2015.10.28.a,2015.10.28,2015.10.28.a
390,2015.07-10,2015.07.10,2015.07.10
560,2014.05.04,2013.05.04,2014.05.04
3522,1967.07.05,1967/07.05,1967.07.05


In [330]:
# Problemas com Case Number
# Linha 390 - Deve receber de Case Number.1
# Linha 3795 - Deve receber de Case Number.1
# Linha 5488 - Deve receber de Case Number.1
# Linha 5944 - Deve receber de Case Number.1
# Linha 6009 - Deve receber de Case Number.1

df2.loc[390,'Case Number']=df2.loc[390,'Case Number.1']
df2.loc[3795,'Case Number']=df2.loc[3795,'Case Number.1']
df2.loc[5488,'Case Number']=df2.loc[5488,'Case Number.1']
df2.loc[5944,'Case Number']=df2.loc[5944,'Case Number.1']
df2.loc[6009,'Case Number']=df2.loc[6009,'Case Number.1']

In [331]:
# Agora que as divergências entre as "Case Number" foram tratadas, as colunas Case Number.1 e Case Number.2 são droppadas

df2=df2.drop(['Case Number.1','Case Number.2'],axis=1)

In [332]:
# Analisando a coluna "Year"

df2['Year'].value_counts()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1801.0      1
1638.0      1
1834.0      1
1723.0      1
1786.0      1
Name: Year, Length: 249, dtype: int64

In [333]:
# Identificando as ocorrências sem ano e incluindo essa informação a partir da informado por "Case Number"

mask=df2['Year'].isnull()
df2.loc[mask,:]
df2.loc[187,'Year']=2017
df2.loc[6079,'Year']=1836

In [334]:
# Trocando o tipo de "Year" para inteiro

df2['Year']=df2['Year'].astype({'Year':'int64'})

In [335]:
# Checando o significado de 0 para essa coluna 

df2.loc[df['Year']==0,:]

# Conclusão: 0 em Year representa registros em que há incerteza da data

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
6177,0000.0214,Ca. 214 B.C.,0,Unprovoked,NaN,Ionian Sea,NaN,Ascending from a dive,"Tharsys, a sponge diver",M,NaN,"FATAL, shark/s bit him in two",Y,NaN,NaN,"Reported by Greek poet, Leonidas of Tarentum (..."
6178,0000.0336,Ca. 336.B.C..,0,Unprovoked,GREECE,Piraeus,In the haven of Cantharus,Washing his pig in preparation for a religious...,A candidate for initiation,M,NaN,"FATAL, shark ""bit off all lower parts of him u...",Y,NaN,NaN,Plutarch (45 - 125 A.D.) in Life of Phoecion (...
6179,0000.0493,493 B.C.,0,Sea Disaster,GREECE,Off Thessaly,NaN,Shipwrecked Persian Fleet,males,M,NaN,Herodotus tells of sharks attacking men in the...,Y,NaN,NaN,Herodotus (485 - 425 B.C.)
6180,0000.0725,Ca. 725 B.C.,0,Sea Disaster,ITALY,Tyrrhenian Sea,Krater found during excavations at Lacco Ameno...,Shipwreck,males,M,NaN,Depicts shipwrecked sailors attacked by a sha...,Y,NaN,NaN,"V.M. Coppleson (1958), p.262, et al"
6181,ND-0154,Before 1939,0,Unprovoked,CANADA,NaN,Grand Banks,Fishing,Joe Folsom,M,NaN,Arm bitten,N,NaN,NaN,"C.E.Russell, pp. 310-311"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6298,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6299,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6300,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [336]:
# Analisando os nulos na coluna "Country" 

mask=df2['Country'].isnull()
df2.loc[mask,:]

# Conclusão: Valores nulos em Country significam que não há informações ou que o ataque 
# aconteceu no mar/em território indefinido

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
62,2017.11.13.R,Reported 13-Nov-2017,2017,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,M,24,Puncture wounds to feet,N,NaN,NaN,Instagram
525,2014.08.00,Aug-2014,2014,Invalid,NaN,NaN,NaN,Sea disaster,Cuban refugees,M,NaN,Shark involvement prior to death not confirmed,NaN,NaN,Shark involvement not confirmed,"Associated Press, 11/27/2014"
2956,1983.00.00.d,Ca. 1983,1983,Unprovoked,NaN,English Channel,NaN,Swimming,Padma Shri Taranath Narayan Shenoy,M,NaN,Left leg bitten,N,NaN,NaN,"Times of India, 2/5/2012"
3378,1970.11.00,Nov-1970,1970,Unprovoked,NaN,NaN,NaN,NaN,Heinz Plotsky,M,NaN,Extensive injuries,N,NaN,NaN,"H.D. Baldridge (1994), SAF Case #1645"
3387,1970.08.02,02-Aug-1970,1970,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,NaN,"Sharks scavenged on bodies, but no record of t...",NaN,Afternoon,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970"
3388,1970.07.05,5-Jul-1970,1970,Unprovoked,NaN,NaN,NaN,NaN,male,M,NaN,Finger or toe severed,N,Night,Mako shark,"H.D. Baldridge (1994), SAF Case #1628"
3395,1970.04.00.b,Apr-1970,1970,Provoked,NaN,NaN,NaN,Freediving,Lionel Jarvis,M,NaN,Arm abraded & lacerated. Recorded as PROVOKED ...,N,NaN,Wobbegong shark,"H.D. Baldridge (1994), SAF Case #1616"
3399,1970.02.05,5-Feb-1970,1970,Unprovoked,NaN,NaN,NaN,Wading,Sally Anne Irvine,F,8,Lacerations to lower leg,N,NaN,Carpet shark,H.D. Baldridge (1994) SAF Case #1626
3425,1969.08.00,Aug-1969,1969,Unprovoked,NaN,NaN,NaN,NaN,Rodney Hughes,M,25,Am lacerated,N,NaN,NaN,H.D. Baldridge (1994) SAF Case #1602
3605,1965.10.21,21-Oct-1965,1965,Unprovoked,NaN,NaN,Florida Strait,The boat Caribou II sank,Mario Castellanos,M,39,Survived,N,NaN,NaN,"Lodi News Sentinel, 10/30/1965"


In [337]:
# Analisando a coluna de data

df2['Date'].value_counts()

# Conclusão: Coluna de data está bastante bagunçada, é melhor tentar criar uma coluna de mês a partir de "Case Number",
# já que data não é um nível de granularidade que dificilmente será interessante numa análise numa base de dados
# com tantos anos.

1957           11
1942            9
1956            8
1958            7
1950            7
               ..
19-Sep-1999     1
16-Jun-2009     1
15-Dec-1936     1
04-Aug-1997     1
22-Jul-2015     1
Name: Date, Length: 5433, dtype: int64

In [338]:
# Criando a função para colocar no apply e separar o mês dentro da coluna "Case Number"

def check_date(item):
    pattern = '\d+\.\d+\.\d+'
    month='\d+\.(\d+)\.\d+'
    if re.findall(pattern,item)!=[]:
        return re.findall(month,item)[0]
    else:
        return 0

In [339]:
df2['Month']=df2['Case Number'].apply(check_date)

In [340]:
df2.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Month
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",06
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",06
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",06
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",06
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,06


In [341]:
df2.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'Month'],
      dtype='object')

In [342]:
# Criando uma coluna de contador

df2['Counter']=1

In [343]:
# Criando uma coluna de "id"

df2['id']=df2.index

In [344]:
# Reordenando as colunas

df2=df2.loc[:,['id','Year','Month','Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source','Case Number', 'Date','Counter']]

In [345]:
df2

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
0,0,2018,06,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,25-Jun-2018,1
1,1,2018,06,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,18-Jun-2018,1
2,2,2018,06,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,09-Jun-2018,1
3,3,2018,06,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,08-Jun-2018,1
4,4,2018,06,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,04-Jun-2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,6297,0,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND.0005,Before 1903,1
6298,6298,0,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND.0004,Before 1903,1
6299,6299,0,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND.0003,1900-1905,1
6300,6300,0,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND.0002,1883-1889,1


In [346]:
# Análise da coluna de "Type"

df2['Type'].value_counts()

# Conclusão: Uniformização dos tipos "Boat", "Boating" e "Boatomg" como "Boating"

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [347]:
mask=df2['Type']=='Boatomg'
df2.loc[mask,'Type']='Boat'

In [348]:
mask=df2['Type']=='Boat'
df2.loc[mask,'Type']='Boating'

In [349]:
# Retirando o espaço extra da coluna "Sex"

df2.rename(columns={'Sex ':'Sex'},inplace=True)

In [350]:
df2.columns

Index(['id', 'Year', 'Month', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'Case Number', 'Date', 'Counter'],
      dtype='object')

In [351]:
# Analisando a coluna "Sex"

df2['Sex'].value_counts()

M      5094
F       637
N         2
M         2
lli       1
.         1
Name: Sex, dtype: int64

In [352]:
# Substituindo 'lli' por 'M' (infomação obtida por meio do nome da pessoa)

mask=df2['Sex']=='lli'
df2.loc[mask,'Sex']='M'

In [353]:
# Substituindo 'N' por 'M' (infomação obtida por meio do nome da pessoa)

mask=df2['Sex']=='N'
df2.loc[mask,'Sex']='M'

In [354]:
# Substituindo '.' por 'Undefined' (não houve como obter a informação)

mask=df2['Sex']=='.'
df2.loc[mask,'Sex']="Undefined"

In [355]:
# Retirando o espaço extra após 'M'

mask=df2['Sex']=='M '
df2.loc[mask,'Sex']='M'

In [356]:
# Analisando a coluna de idade

df2['Age'].value_counts()


17         154
18         150
19         142
20         141
15         139
          ... 
F            1
Teens        1
X            1
6½           1
34 & 19      1
Name: Age, Length: 157, dtype: int64

In [357]:
# Criando uma função para remover espaços indesejados no início e no final da idade

def remove_spaces(word):
    try:
        word=word.strip()
        return word
    except:
        return word

In [358]:
df2['Age']=df2['Age'].apply(remove_spaces)

In [359]:
# Substituir as informações não numéricas por "Inaccurate information"

mask=df2['Age'].str.isnumeric()==False
df2.loc[mask,'Age']="Inaccurate information"

In [360]:
# Analisando a coluna de fatalidade

df2['Fatal (Y/N)'].value_counts()

# Conclusão: Remover espaços, uniformizar demais informações

N          4293
Y          1388
UNKNOWN      71
 N            7
y             1
2017          1
N             1
M             1
Name: Fatal (Y/N), dtype: int64

In [361]:
# Removendo espaços indesejados

df2['Fatal (Y/N)']=df2['Fatal (Y/N)'].apply(remove_spaces)

In [362]:
# Uniformizando todos os 'y' como maiúsculos

mask=df2['Fatal (Y/N)']=='y'
df2.loc[mask,'Fatal (Y/N)']="Y"

In [363]:
# Substituindo informações que não são idade por "UNKNOWN"

mask=df2['Fatal (Y/N)']=='2017'
df2.loc[mask,'Fatal (Y/N)']="UNKNOWN"

In [364]:
mask=df2['Fatal (Y/N)']=='M'
df2.loc[mask,'Fatal (Y/N)']="UNKNOWN"

In [366]:
# Nomeando 0 em Year como "Unknown" 

df2.loc[df2['Year']==0,'Year']='Unknown'

In [367]:
# Nomeando 0 em Year como "Unknown" 

df2.loc[df2['Month']==0,'Month']='Unknown'

In [368]:
# Nomeando 00 em Year como "Unknown" 

df2.loc[df2['Month']=='00','Month']='Unknown'

In [369]:
# Nomeando nulos de Age como "Without information"

df2.loc[df2['Age'].isnull(),'Age']='Without information'

In [370]:
# Nomear nulos de Sex como "Without information"

df2.loc[df2['Sex'].isnull(),'Sex']='Without information'

In [371]:
# Nomear nulos de Fatal como "Without information"

df2.loc[df2['Fatal (Y/N)'].isnull(),'Fatal (Y/N)']='Without information'

In [372]:
df2.head()

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
0,0,2018,06,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,25-Jun-2018,1
1,1,2018,06,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,18-Jun-2018,1
2,2,2018,06,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,09-Jun-2018,1
3,3,2018,06,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,Without information,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,08-Jun-2018,1
4,4,2018,06,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,Without information,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,04-Jun-2018,1


In [373]:
# Tratando os países com / a partir da informação da coluna Area

mask=df2['Country'].str.contains('/')==True
df2.loc[mask,:]

# 2292 - South Sinai = Egito
# 3957 - Aden gulf = Yemen
# 4140 - India
# 4619 - Shatt-al-Arab River = Iraq
# 4696 - Solomon Islands
# 4934 - Kralievica = Croatia
# 5978 - Fernando Po Island = Equatorial Guinea

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
2292,2292,1996,07,Unprovoked,EGYPT / ISRAEL,"South Sinai, Gulf of Aqaba","1 km off the mouth of Marsa Bereika, north of ...",Free diving with a pod of dolphins,Martin Christopher Richardson,M,Without information,"Bitten on back, shoulder & chest",N,18h00,said to involve an oceanic whitetip shark,"Daily Telegraph, 7/26/1996, p.15; K. Lavalli &...",1996.07.23.a,23-Jul-1996,1
3957,3957,1960,10,Sea Disaster,RED SEA / INDIAN OCEAN,Enroute from Suez to Aden (Yemen),NaN,Cargo ship El Gamil entroute Suez to Yemen (Ad...,NaN,M,Without information,"Sharks attacked sailors in the water, several ...",Y,NaN,NaN,"V.M. Coppleson (1962), p.260",1960.10.00.d,Oct-1960,1
4140,4140,1958,09,Unprovoked,ANDAMAN / NICOBAR ISLANDAS,NaN,NaN,"""Climbing up to ship after repairing the stern...",Sailor of tuna vessel Daisan-Tenyo-Maru,M,32,"FATAL, leg bitten",Y,14h00,Blue shark,M. Hosina,1958.09.13,13-Sep-1958,1
4619,4619,1945,08,Unprovoked,IRAN / IRAQ,Shatt-al-Arab River,Vicinity of Abadan,NaN,Abdol Karmi,M,Without information,Lower left leg amputated,N,NaN,NaN,"A. Anderson, M.D. / Lt. Col. R.S. Hunt, Royal ...",1945.08.19,19-Aug-1945,1
4696,4696,1942,11,Sea Disaster,SOLOMON ISLANDS / VANUATU,NaN,"North of Guadalcanal, Solomon Islands while en...",Explosion & sinking of the USS Juneau after be...,Because of a mistaken belief that there were n...,M,Without information,Over a period of a week men in the water died ...,Y,11h01 -time of ship sinking,NaN,US Navy Military History,1942.11.13.b,13-Nov-1942,1
4934,4934,1934,08,Invalid,ITALY / CROATIA,NaN,Kralievica,Swimming,Zorca Prinz,F,Without information,Reported to be FATAL and involve a 6m white s...,Without information,NaN,Invalid,"C. Moore & R. Roccini, GSAF; D. Baldridge, pp....",1934.08.26.R,Reported 26-Aug-1934,1
5978,5978,1861,02,Unprovoked,EQUATORIAL GUINEA / CAMEROON,Fernando Po Island,NaN,Swimming,William Looney,M,Without information,FATAL,Y,NaN,NaN,"Daily Southern Cross, 2/12/1861",1861.02.12.R,Reported 12-Feb-1861,1


In [374]:
# Tratando os países não encontrados pelo Tableau

# 2292 - South Sinai = Egito
# 3957 - Aden gulf = Yemen
# 4140 - India
# 4619 - Shatt-al-Arab River = Iraq
# 4696 - Solomon Islands
# 4934 - Kralievica = Croatia
# 5978 - Fernando Po Island = Equatorial Guinea

df2.loc[2292,'Country']='EGYPT'
df2.loc[3957,'Country']='YEMEN'
df2.loc[4140,'Country']='INDIA'
df2.loc[4619,'Country']='IRAQ'
df2.loc[4696,'Country']='SOLOMON ISLANDS'
df2.loc[4934,'Country']='CROATIA'
df2.loc[5978,'Country']='EQUATORIAL GUINEA'

In [375]:
mask=df2['Country'].str.contains(',')==True
df2.loc[mask,:]

# ST HELENA, British overseas territory = ST HELENA

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
111,111,2017,07,Unprovoked,"ST HELENA, British overseas territory",Ascension Island,English Bay,Surfing,Kawika Matsu,M,37,Torso bitten,N,17h00,NaN,"Telegraph UK, 7/28/2017",2017.07.24,24-Jul-2017,1
152,152,2017,04,Unprovoked,"ST HELENA, British overseas territory",Ascension Island,English Bay,Snorkeling,Frankie Gonsalves,F,40,Calf & foot bitten,N,16h00,NaN,"Daily Mail, 4/25/2017",2017.04.22,22-Apr-2017,1


In [376]:
df2.loc[mask,'Country']='ST HELENA'

In [377]:
# Removendo o ? da coluna 'Country'

mask=df2['Country'].str.contains('\?')==True
df2.loc[mask,:]

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
4345,4345,1954,04,Provoked,SUDAN?,Red Sea,Southern part,Spearfishing,Jean Foucher-Createau,M,Without information,"Speared small shark, shark bit his thigh and/o...",N,NaN,NaN,"V.M. Coppleson (1962), p.254; H.D. Baldridge, ...",1954.04.00,Apr-1954,1
5894,5894,1872,11,Unprovoked,INDIAN OCEAN?,NaN,NaN,Swimming to avoid capture,Malay pirates,M,Without information,FATAL,Y,NaN,NaN,"The Mercury, 11/230/1872",1872.11.30.R,Reported 30-Nov-1872,1
6228,6228,Unknown,Unknown,Unprovoked,RED SEA?,NaN,NaN,Free diving,Jill Reed,F,Without information,"Shoulder scratched, swim fin bitten",N,Morning,NaN,"Captain T. Falcon-Barker, pp. 91-93",ND.0085,"No date, Before 1969",1
6291,6291,Unknown,Unknown,Unprovoked,ASIA?,NaN,NaN,Swimming,Mr. Masury,M,Without information,Foot severed,N,NaN,NaN,"Ref. J. T. Dubois in N.Y. Sun, 3/19/1911",ND.0011,Before 1911,1


In [378]:
df2.loc[4345,'Country']='SUDAN'
df2.loc[5894,'Country']='INDIAN OCEAN'
df2.loc[6228,'Country']='RED SEA'
df2.loc[6291,'Country']='ASIA'

In [379]:
mask=df2['Country'].str.contains('Between')==True
df2.loc[mask,:]

# 6170 = Portugal

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
6170,6170,1580,01,Unprovoked,Between PORTUGAL & INDIA,NaN,NaN,Man fell overboard from ship. Those on board t...,male,M,Without information,"FATAL. ""Shark tore him to pieces.",Y,NaN,NaN,"G.P. Whitley, p. 10",1580.01.10.R,Letter dated 10-Jan-1580,1


In [380]:
df2.loc[6170,'Country']='PORTUGAL'

In [381]:
mask=df2['Country'].str.contains('^Coast')==True
df2.loc[mask,:]

# 6059 = São Tomé Island

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
6059,6059,1844,05,Unprovoked,Coast of AFRICA,Island of St. Thomas,NaN,Fell overboard,"Martin, coxswain of the USS Saratoga",M,Without information,FATAL,Y,NaN,NaN,"Tioga Eagle, 7/31/1844",1844.05.24,24-May-1844,1


In [382]:
df2.loc[6059,'Country']='ST THOMAS ISLAND'

In [383]:
mask=df2['Country'].str.contains('^COLUMBIA')==True
df2.loc[mask,:]

# COLUMBIA = COLOMBIA

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
257,257,2016,06,Unprovoked,COLUMBIA,Isla Provedencia,NaN,Scuba Diving,Arturo Velez,M,59,Severe bite to right hand,N,11h00,"Caribbean reef shark, 4.5'",Dr. A. Velez,2016.06.24,24-Jun-2016,1
901,901,2011,07,Unprovoked,COLUMBIA,Sucre,"Libertad, San Onofre",Kayaking,Andrés Tulio Amaya Vidal,M,17,"Right arm bitten, defense wounds to left hand",N,10h30,NaN,"El Universal, 7/6/2011",2011.07.05,05-Jul-2011,1
913,913,2011,06,Unprovoked,COLUMBIA,San Andrés archipelago,Albuquerque Cay,Spearfishing,Jhon Jairo James,M,24,Injuries to right hand and forearm,N,NaN,NaN,"Il Isleno.com, 6/6/2011",2011.06.06.R,Reported 06-Jun-2011,1
3463,3463,1968,08,Unprovoked,COLUMBIA,Magdalena Department,"Makuaka Caño, Taganga",Dynamite fishing,Abel Mattos Antoniou Vasquez,M,18,Lacerations to head,N,10h30,2 m shark,"SoHo.com, 11/19/2-11",1968.08.08,08-Aug-1968,1
3516,3516,1967,08,Unprovoked,COLUMBIA,NaN,NaN,NaN,Thomas Walsh,M,Without information,3 fingers were bitten by a shark,N,NaN,NaN,"New York Times, 8/14/1967",1967.08.14.R,Reported 14-Aug-1967,1
3591,3591,1966,01,Sea Disaster,COLUMBIA,NaN,Off Cartagena,Colombian (Avianca) DC-4 airliner plunged int...,NaN,Without information,Without information,"10 survived, 51 perished.",Y,NaN,Shark involvement not confirmed,"Virgin Islands Daily News, 1/17/1966; SAF Case...",1966.01.14,14-Jan-1966,1
3937,3937,1961,01,Unprovoked,COLUMBIA,Roncador Bank,"Roncador Bank, 135 nm north of San Andres",Taking boat from California to Florida when it...,Joe Chaney,M,Without information,Leg bitten,N,NaN,1.8 m [6'] shark,V.M. Coppleson (1962) p.246,1961.01.01,01-Jan-1961,1
4314,4314,1955,Unknown,Unprovoked,COLUMBIA,Isles del Rosario,Southwest of Cartegena,Spearfishing,Gabriel Echiavarria,M,12,Swim fin & foot bitten,N,NaN,1.5 m [5'] shark,"H. Echavarria; H.D. Baldridge, p.135; M. McDia...",1955.00.00.b,Ca. 1955,1
4439,4439,1951,08,Unprovoked,COLUMBIA,Caribbean Sea,Cartegena,Bathing,"4 people killed in the past week, others injured",Without information,Without information,FATAL,Y,NaN,NaN,"New York Times, 8/9/1951; V.M. Coppleson (1958...",1951.08.00,Between 01-Aug-1951 & 08-Aug-1951,1


In [384]:
mask=df2['Country'].str.contains('^COLUMBIA')==True
df2.loc[mask,'Country']='COLOMBIA'

In [385]:
mask=df2['Country'].str.contains('^DIEGO')==True
df2.loc[mask,:]

# DIEGO GARCIA = UNITED KINGDOM

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
655,655,2013,07,Unprovoked,DIEGO GARCIA,NaN,NaN,Swimming,Fernando Licay,M,33,FATAL,Y,NaN,NaN,"Sun Star, 7/18/2013",2013.07.14,14-Jul-2013,1


In [386]:
df2.loc[655,'Country']='UNITED KINGDOM'

In [387]:
mask=df2['Country'].str.contains('^KOREA')==True
df2.loc[mask,:]

# KOREA = SOUTH KOREA

,id,Year,Month,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number,Date,Counter
6220,6220,Unknown,Unknown,Unprovoked,KOREA,South Korea,Cheju Island,Diving,"female, a Hae Nyeo",F,Without information,"FATAL, injured while diving, then shark bit her",Y,NaN,NaN,"K. Amsler, Divernet.com",ND.0094,"No date, Before May-1996",1


In [388]:
df2.loc[6220,'Country']='SOUTH KOREA'

In [389]:
# ADMIRALTY ISLANDS = USA
# ANDAMAN ISLANDS = INDIA
# BRITISH NEW GUINEA = Papua New Guinea
# CEYLON AND CEYLON(SRI LANKA) = SRI LANKA  
# CRETE = GREECE
# JAVA = INDONESIA
# JOHNSTON ISLAND = USA
# NEVIS = Saint Kitts and Nevis
# NEW BRITAIN = Papua New Guinea
# NEW GUINEA = Papua New Guinea
# OKINAWA = JAPAN
# ROATAN = HONDURAS
# SAN DOMINGO = DOMINICAN REPUBLIC
# ST. MAARTEN = NETHERLANDS
# TRATA NULOS

In [390]:
df2.loc[df2['Country'].str.contains('^ADMIRALTY ISLANDS')==True,'Country']='USA'
df2.loc[df2['Country'].str.contains('^ANDAMAN ISLANDS')==True,'Country']='INDIA'
df2.loc[df2['Country'].str.contains('^BRITISH NEW GUINEA')==True,'Country']='PAPUA NEW GUINEA'
df2.loc[df2['Country'].str.contains('^CEYLON')==True,'Country']='SRI LANKA'
df2.loc[df2['Country'].str.contains('^CRETE')==True,'Country']='GREECE'
df2.loc[df2['Country'].str.contains('^JAVA')==True,'Country']='INDONESIA'
df2.loc[df2['Country'].str.contains('^JOHNSTON ISLAND')==True,'Country']='USA'
df2.loc[df2['Country'].str.contains('^NEVIS')==True,'Country']='SAINTS KITTS AND NEVIS'
df2.loc[df2['Country'].str.contains('^NEW BRITAIN')==True,'Country']='PAPUA NEW GUINEA'
df2.loc[df2['Country'].str.contains('^NEW GUINEA')==True,'Country']='PAPUA NEW GUINEA'
df2.loc[df2['Country'].str.contains('^OKINAWA')==True,'Country']='JAPAN'
df2.loc[df2['Country'].str.contains('^ROATAN')==True,'Country']='HONDURAS'
df2.loc[df2['Country'].str.contains('^SAN DOMINGO')==True,'Country']='DOMINICAN REPUBLIC'
df2.loc[df2['Country'].str.contains('^ST. MAARTEN')==True,'Country']='NETHERLANDS'

In [391]:
# Preenchendo as células sem país com "UNKNOWN"

df2.loc[df2['Country'].isnull()==True,'Country']='UNKNOWN'

In [392]:
# Criando um .csv com a base limpa

df2.to_csv('shark_attack-cleaned.csv', index=False)